In [4]:
#import tensorflow as tf
import numpy as np
import math
import glob
import matplotlib

import matplotlib.pyplot as plt
from matplotlib.pyplot import plot,savefig
from dataset import *

from sklearn.datasets import load_iris
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
import mylib as ml2
from xgboost import XGBClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
import pandas as pd


/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [5]:
#------------------------read data----------------------------#
cutsize = 256
circle_num = 8
expansion_data_number = 500
noise_scales = 2
"""
dataset_file_name = 'dataset_' + str(circle_num) + '_' + str(cutsize) + '.npy'
label_file_name = 'label_' + str(circle_num) + '_' + str(cutsize) + '.npy'
"""
"""
data_path = '/home/silver-bullet/newpaper/data/dataset/'
dataset, label = load_dataset(data_path, circle_num=circle_num, cutsize=cutsize) 
"""
data_path = '/home/silver-bullet/newpaper/data/CWRUdataset'
dataset, label = load_CWRU_data(data_path)


if circle_num>=9:
    dataset, label = expansion_and_add_noise(dataset,label,exnumber=expansion_data_number, noise_scales=noise_scales)



x_number = dataset.shape[0]

feature_data = feature_extractor(dataset)
print('feature_data.shape: ', feature_data.shape)

x = feature_data.reshape(x_number,-1)
print('x.shape: ',x.shape)
y = label.reshape(x_number,)
print('y.shape: ', y.shape)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=56)

print('X_train.shape: ', X_train.shape)
print('X_test.shape: ',X_test.shape)
print('y_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

dataset shape :  (832, 1024, 2)
label shape :  (832, 1)
N: 1024
sum_square.shape():  (832, 2)
RMS.shape():  (832, 2)
data_mean.shape():  (832, 2)
data_std.shape():  (832, 2)
d_data_mean.shape:  (832, 1024, 2)
Variance.shape:  (832, 2)
Skewness.shape:  (832, 2)
Kurtosis.shape:  (832, 2)
SF.shape:  (832, 2)
CF.shape:  (832, 2)
IF.shape:  (832, 2)
MF.shape:  (832, 2)
diff1_data.shape:  (832, 1023, 2)
diff2_data.shape:  (832, 1022, 2)
std_diff1.shape:  (832, 2)
std_diff2.shape:  (832, 2)
mobility.shape:  (832, 2)
complexity.shape:  (832, 2)
FC.shape:  (832, 2)
MSF.shape:  (832, 2)
RMSF.shape:  (832, 2)
RVF.shape:  (832, 2)
feature_data.shape:  (832, 19, 2)
x.shape:  (832, 38)
y.shape:  (832,)
X_train.shape:  (665, 38)
X_test.shape:  (167, 38)
y_train.shape:  (665,)
y_test.shape:  (167,)


In [6]:
#---------------------XGBoost----------------------------#
"""
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 5,
    'gamma': 0.1,
    'max_depth': 8,
    'lambda': 2,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'silent': 1,
    'eta': 0.1,
    'seed': 1000,
    'nthread': 4,
}

plst = params.items()

dtrain = xgb.DMatrix(X_train, y_train)
num_rounds = 1000
model = xgb.train(plst, dtrain, num_rounds)
"""
model = XGBClassifier(learning_rate=0.1,
                        n_estimators=500,         # 树的个数--1000棵树建立xgboost
                        max_depth=16,               # 树的深度
                        min_child_weight = 1,      # 叶子节点最小权重
                        gamma=0.1,                  # 惩罚项中叶子结点个数前的参数
                        subsample=0.8,             # 随机选择80%样本建立决策树
                        colsample_btree=0.8,       # 随机选择80%特征建立决策树
                        objective='multi:softmax', # 指定损失函数
                        scale_pos_weight=1,        # 解决样本个数不平衡的问题
                        )

model.fit(X_train,y_train)
# 对测试集进行预测
#dtest = xgb.DMatrix(X_test)
#ans = model.predict(dtest)
ans = model.predict(X_test)
print('ans.shape: ', ans.shape)

ml2.evaluate_model2(ans, y_test)

# 计算准确率
cnt1 = 0
cnt2 = 0

for i in range(len(y_test)):
    if ans[i] == y_test[i]:
        cnt1 += 1
    else:
        cnt2 += 1
print("Accuracy: %.2f %% " % (100 * cnt1 / (cnt1 + cnt2)))


ans.shape:  (167,)
normal result:
tp: 42  tn: 0  fp: 0  fn: 125
tp/(tp+tn):  1.0
tp/(tp+fp):  1.0
inner_ring result:
tp: 23  tn: 0  fp: 0  fn: 144
tp/(tp+tn):  1.0
tp/(tp+fp):  1.0
outer_ring result:
tp: 22  tn: 0  fp: 0  fn: 145
tp/(tp+tn):  1.0
tp/(tp+fp):  1.0
roller result:
tp: 30  tn: 0  fp: 0  fn: 137
tp/(tp+tn):  1.0
tp/(tp+fp):  1.0
joint result:
tp: 23  tn: 0  fp: 0  fn: 144
tp/(tp+tn):  1.0
tp/(tp+fp):  1.0
Accuracy: 100.00 % 


In [7]:
#---------------------------K Fold-------------------------------------------#
scoring = ['precision_macro', 'recall_macro']
scores = cross_validate(model, X_train, y_train, scoring=scoring,cv=10, return_train_score=True)
sorted(scores.keys())
print('test result:')
print(scores)
scores_df = pd.DataFrame(scores)

table_header = ['test_recall_macro', 'train_recall_macro','fit_time', 'train_precision_macro','test_precision_macro']

scores_df

test result:
{'train_recall_macro': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), 'test_precision_macro': array([1.        , 1.        , 0.99206349, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ]), 'train_precision_macro': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), 'fit_time': array([1.20431733, 0.89592385, 0.85564017, 0.85766339, 0.86387229,
       0.84172654, 0.86490202, 0.84350157, 0.82794499, 0.84486842]), 'score_time': array([0.0054121 , 0.00575161, 0.00665784, 0.00492334, 0.00539732,
       0.00521207, 0.00522161, 0.00613785, 0.00447011, 0.00456667]), 'test_recall_macro': array([1.        , 1.        , 0.98148148, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ])}


,fit_time,score_time,test_precision_macro,test_recall_macro,train_precision_macro,train_recall_macro
0,1.204317,0.005412,1.000000,1.000000,1.0,1.0
1,0.895924,0.005752,1.000000,1.000000,1.0,1.0
2,0.855640,0.006658,0.992063,0.981481,1.0,1.0
3,0.857663,0.004923,1.000000,1.000000,1.0,1.0
4,0.863872,0.005397,1.000000,1.000000,1.0,1.0
5,0.841727,0.005212,1.000000,1.000000,1.0,1.0
6,0.864902,0.005222,1.000000,1.000000,1.0,1.0
7,0.843502,0.006138,1.000000,1.000000,1.0,1.0
8,0.827945,0.004470,1.000000,1.000000,1.0,1.0
9,0.844868,0.004567,1.000000,1.000000,1.0,1.0
